In [ ]:
import re
sample_file_name = "../dutch-language/data/sample-sentences-nlwiki.txt"
short_file_name = "../dutch-language/data/short-sentences-nlwiki.txt"
full_file_name = "../dutch-language/data/clean-sentences-nlwiki.txt"
corrected_file_name = "../dutch-language/data/corrected-sentences-nlwiki.txt"

sentences_file_name = corrected_file_name
positives_file_name = "../dutch-language/data/positive-sentences-002.txt"

BATCH_SIZE = 500
TOTAL_SIZE = 10000000

regex_nd = re.compile(r'.*')


from simpletransformers.classification import ClassificationModel
import pandas as pd
import logging

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

model = ClassificationModel(
    "roberta", "outputs/RoBERTa-004", use_cuda=True
)
print(type(model))


with open(sentences_file_name, 'r') as sentences_file:
    with open(positives_file_name, 'w') as positives_file:
        total_count = 0
        match_count = 0
        batch = []
        batch_count = len(batch)

        for line in sentences_file:
            # process lines
            clean_line = line.rstrip()
            if (regex_nd.match(clean_line)):
                match_count += 1

                # fill batch
                batch_count += 1
                batch.append(clean_line)
                if batch_count >= BATCH_SIZE:
                    # process batch
                    predictions, raw_outputs = model.predict(batch)
                    for index in range(len(predictions)):
                        is_positive = (predictions[index] == 1) # can be more subtle later; no, maybe, true
                        if (is_positive):
                            positives_file.write(f"1\t{raw_outputs[index][0]}\t{raw_outputs[index][1]}\t{batch[index]}\n")
                    batch = []
                    batch_count = len(batch)

            # total count
            total_count += 1
            if (total_count >= TOTAL_SIZE):
                break


print()
print(total_count)
print(match_count)

In [ ]:
from simpletransformers.classification import ClassificationModel
import pandas as pd
import logging

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

model = ClassificationModel(
    "roberta", "outputs/RoBERTa-003", use_cuda=True
)
print(type(model))


In [ ]:
predictions, raw_outputs = model.predict([
    "Ik word nieuwsgierig.",
    "Hoe word je gevraagd?",
    "Wat wordt je gevraagd?",
    "Wat word je opdringerig, zeg!",

    "Ik zend een pakje.",
    "Ik zend je een pakje.",
    "Ik zend u een heel mooie trui.",
    "Je zendt het toch nog vandaag op, hoop ik.", #
    "Jij zendt een hele grote doos naar oma.",
    "Zend je dat straks uit?", #
    "Zend jij echt die fiets via de post?",
    "Welke baas zendt je nu helemaal naar London voor 1 klant?",

    # With spelling mistake
    "Ik wordt nieuwsgierig.",
    "Hoe wordt je gevraagd?",
    "Wat word je gevraagd?",
    "Wat wordt je opdringerig, zeg!",

    "Ik zendt een pakje.",
    "Ik zendt je een pakje.",
    "Ik zendt u een heel mooie trui.", #
    "Je zend het toch nog vandaag op, hoop ik.",
    "Jij zend een hele grote doos naar oma.",
    "Zendt je dat straks uit?", #
    "Zendt jij echt die fiets via de post?",
    "Welke baas zend je nu helemaal naar London voor 1 klant?",

    # Trying other words
    # Correct spelling
    "Ik vind dit toch niet zo mooi.",
    "Wat vind jij van al die aandacht?",
    "Hoe vind ik nu de ingang?",
    "Het is toch erg dat hij dat niet vindt.",
    "Hoe zwaar vindt hij de opleiding?",

    "Ik loop er zo maar voorbij.",
    "Loop jij ook zo snel?",
    "Je loopt daar beter niet telkens over.",
    "Jij loopt echt helemaal naar zee?",
    "En daarom loopt hij er met een grote bocht omheen.",

    # With spelling mistake
    "Ik vindt dit toch niet zo mooi.",
    "Wat vindt jij van al die aandacht?",
    "Hoe vindt ik nu de ingang?",
    "Het is toch erg dat hij dat niet vind.", # Incorrectly evaluated to "0"
    "Hoe zwaar vind hij de opleiding?",

    "Ik loopt er zo maar voorbij.",
    "Loopt jij ook zo snel?",
    "Je loop daar beter niet telkens over.",
    "Jij loop echt helemaal naar zee?",
    "En daarom loop hij er met een grote bocht omheen."
])
print()
print(predictions)
print(raw_outputs)

# Expected           [0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1]
# Actual RoBERTa-001 [0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 1 1 1 0 1 1 1 1 1 1]
# Actual RoBERTa-002 [0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 1 1 1 0 1 1 0 1 1 1]
# Actual RoBERTa-003 [0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 1 1 1 0 1 1 1 1 1 1]
# Actual    BERT-001 [0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 1 1 0 1 0]



In [ ]:
# Convert the strong positives (fp) to examples negatives
# cat positive-sentences-processed.txt | grep '^1.*\-[34]\.' | sed 's/.*\t\(..*\)$/    ["\1", 0],/' > strong-positives-to-correct.txt